In [3]:
#필요라이브러리 설치
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print('hello')

# 버전 확인
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")

hello
pandas 버전: 2.3.3
numpy 버전: 2.4.0


In [4]:
#현재 작업 경로 확인
import os
print(os.getcwd())

#파일 불러오기
df = pd.read_csv('rd/union1.csv', encoding='utf-8')
print("cp949로 로드 성공!")

c:\Users\Administrator\Documents\workspace\budongsan3_project\ysy
cp949로 로드 성공!


In [ ]:
#데이터 살펴보기

df.head()
df.info()
df.describe()
df.shape
df.columns.tolist()

In [ ]:
#계약일을 datetime형식으로 변환
df['계약일'] = pd.to_datetime(df['계약일'], format= '%Y%m%d')
df['물건금액'] = round(df['물건금액'], 0)

print(df['계약일'].head())
print(df['물건금액'].head())

In [ ]:

#자치구-법정동 기준
trend_detail = df.groupby(['계약일','자치구명','법정동명'])['물건금액'].mean().round(0).unstack(level=['자치구명','법정동명']).sort_index(axis=1)

#자치구별 기준
trend_gu = trend_detail.groupby(level='자치구명', axis=1).mean().round(0)

#전체 강남3구 기준
trend_total = df.groupby(['계약일'])['물건금액'].mean().round(0)  
overall_avg = trend_total.mean().mean()

print(f"전체 강남3구 기준 평균거래금액: {overall_avg:,.2f}만 원")

trend_detail.head()
trend_total.head()

In [ ]:
# 인덱스를 datetime으로 변경⭐⭐⭐ 
# groupby하면서 '계약일'이 인덱스가 되면서 datetime이 풀렸을 가능성이 큼
trend_detail.index = pd.to_datetime(trend_detail.index)
trend_gu.index = pd.to_datetime(trend_gu.index)
trend_total.index = pd.to_datetime(trend_total.index)

#구-동 일별 데이터 ==> 월별로 resampling하기
trend_detail_monthly = trend_detail.resample('MS').mean()

# 구별 데이터를 일별 ==> 월별로 resampling하기
trend_gu_monthly = trend_gu.resample('MS').mean()

# 강남3구 데이터를 일별 ==> 월별로 resampling하기
trend_total_monthly = trend_total.resample('MS').mean()

print(trend_detail_monthly.head())
print(trend_gu_monthly.head())
print(trend_total_monthly.head())

print(f"시구별 데이터 행 개수: {len(trend_detail_monthly)}")
print(f"구별 데이터 행 개수: {len(trend_gu_monthly)}")
print(f"강남3구 전체 평균 행 개수: {len(trend_total_monthly)}")

In [17]:
import pandas as pd
import plotly.graph_objects as go

# 1. 데이터 집계 (기존 로직 유지)
df_2025 = df[df['계약일'].dt.year == 2025].copy()
df_2025['월_숫자'] = df_2025['계약일'].dt.month
df_2025['월'] = df_2025['월_숫자'].astype(str) + "월"

monthly_gu = df_2025.groupby(['월_숫자', '월', '자치구명'])['물건금액'].mean().reset_index()
monthly_total = df_2025.groupby(['월_숫자', '월'])['물건금액'].mean().reset_index()
monthly_total['자치구명'] = '강남3구 전체'
trend_df = pd.concat([monthly_gu, monthly_total]).sort_values('월_숫자')

# [기능 분리] 1. 레이블용 상세 포맷 (억 + 만원)
def format_detailed_won(val):
    uk = int(val // 10000)
    man = int(val % 10000)
    if uk > 0:
        return f"{uk}억 {man:,.0f}만"
    else:
        return f"{man:,.0f}만"

# [기능 분리] 2. Y축 눈금용 포맷 (억 단위만)
def format_only_uk(val):
    return f"{int(val // 10000)}억"

# 2. 선 그래프 생성
fig = go.Figure()

colors = {'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen', '강남3구 전체': 'red'}
widths = {'강남구': 2.0, '서초구': 2.0, '송파구': 2.0, '강남3구 전체': 3.75}

for name in ['강남구', '서초구', '송파구', '강남3구 전체']:
    target_data = trend_df[trend_df['자치구명'] == name]
    
    fig.add_trace(go.Scatter(
        x=target_data['월'],
        y=target_data['물건금액'],
        name=name,
        mode='lines+markers',
        line=dict(color=colors[name], width=widths[name]),
        marker=dict(size=8),
        hoverinfo='name+x+y'
    ))

    # 최고/최저점 데이터 레이블 (세밀하게 표시)
    max_row = target_data.loc[target_data['물건금액'].idxmax()]
    min_row = target_data.loc[target_data['물건금액'].idxmin()]

    for row, pos in [(max_row, 'top'), (min_row, 'bottom')]:
        fig.add_annotation(
            x=row['월'],
            y=row['물건금액'],
            text=format_detailed_won(row['물건금액']), # 상세 단위 사용
            showarrow=False,
            yshift=12 if pos == 'top' else -12,
            font=dict(color=colors[name], size=10)
        )

# 3. Y축 눈금 설정 (억 단위로 표시)
max_val = trend_df['물건금액'].max()
min_val = 100000 # 10억
tick_vals = [i for i in range(min_val, int(max_val) + 50000, 50000)]
tick_texts = [format_only_uk(v) for v in tick_vals] # 억 단위만 사용

# 4. 레이아웃 설정
fig.update_layout(
    title=dict(
        text='<b>2025년 강남 3구 지역별/전체 월별 실거래가 추이</b>',
        x=0.5, y=0.88, font=dict(size=22)
    ),
    template='plotly_white',
    width=1100, height=750,
    margin=dict(l=100, r=50, t=140, b=100),
    xaxis=dict(title=None, showgrid=False),
    yaxis=dict(
        title="평균 실거래가 (Average Transaction Price)",
        tickmode='array',
        tickvals=tick_vals,
        ticktext=tick_texts, # Y축은 깔끔하게 억 단위
        showgrid=True,
        gridcolor='rgba(200, 200, 200, 0.3)',
        gridwidth=1,
        griddash='dot',
        range=[min_val, max_val * 1.15]
    ),
    legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="center", x=0.5)
)

fig.show()

In [8]:
import pandas as pd
import plotly.graph_objects as go

# 1. 데이터 집계 (2023년 ~ 2025년으로 확장)
# 연도 필터링 범위를 2023년부터 2025년까지로 수정
df_period = df[(df['계약일'].dt.year >= 2023) & (df['계약일'].dt.year <= 2025)].copy()

# 시계열 순서 정렬을 위해 '연월' 컬럼 생성 (예: 2023-01)
df_period['연월_순서'] = df_period['계약일'].dt.to_period('M')
df_period['연월_표시'] = df_period['계약일'].dt.strftime('%Y-%m')

# 그룹화 (연월 순서대로 정렬하기 위해 '연월_순서' 포함)
monthly_gu = df_period.groupby(['연월_순서', '연월_표시', '자치구명'])['물건금액'].mean().reset_index()
monthly_total = df_period.groupby(['연월_순서', '연월_표시'])['물건금액'].mean().reset_index()
monthly_total['자치구명'] = '강남3구 전체'

trend_df = pd.concat([monthly_gu, monthly_total]).sort_values('연월_순서')

# [기능 분리] 1. 레이블용 상세 포맷 (억 + 만원)
def format_detailed_won(val):
    uk = int(val // 10000)
    man = int(val % 10000)
    if uk > 0:
        return f"{uk}억 {man:,.0f}만"
    else:
        return f"{man:,.0f}만"

# [기능 분리] 2. Y축 눈금용 포맷 (억 단위만)
def format_only_uk(val):
    return f"{int(val // 10000)}억"

# 2. 선 그래프 생성
fig = go.Figure()

colors = {'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen', '강남3구 전체': 'red'}
widths = {'강남구': 2.0, '서초구': 2.0, '송파구': 2.0, '강남3구 전체': 3.75}

for name in ['강남구', '서초구', '송파구', '강남3구 전체']:
    target_data = trend_df[trend_df['자치구명'] == name]
    
    fig.add_trace(go.Scatter(
        x=target_data['연월_표시'], # X축을 연도-월로 변경
        y=target_data['물건금액'],
        name=name,
        mode='lines+markers',
        line=dict(color=colors[name], width=widths[name]),
        marker=dict(size=6), # 데이터 포인트가 많아지므로 사이즈 소폭 조절
        hoverinfo='name+x+y'
    ))

    # 전체 기간 중 최고/최저점 데이터 레이블
    max_row = target_data.loc[target_data['물건금액'].idxmax()]
    min_row = target_data.loc[target_data['물건금액'].idxmin()]

    for row, pos in [(max_row, 'top'), (min_row, 'bottom')]:
        fig.add_annotation(
            x=row['연월_표시'],
            y=row['물건금액'],
            text=format_detailed_won(row['물건금액']),
            showarrow=False,
            yshift=12 if pos == 'top' else -12,
            font=dict(color=colors[name], size=9) # 가독성을 위해 폰트 사이즈 소폭 조절
        )

# 3. Y축 눈금 설정
max_val = trend_df['물건금액'].max()
min_val = 100000 # 10억
tick_vals = [i for i in range(min_val, int(max_val) + 100000, 50000)]
tick_texts = [format_only_uk(v) for v in tick_vals]

# 4. 레이아웃 설정
fig.update_layout(
    title=dict(
        text='<b>2023-2025 강남 3구 지역별/전체 월별 실거래가 추이</b>',
        x=0.5, y=0.92, font=dict(size=22)
    ),
    template='plotly_white',
    width=1300, height=750, # 데이터가 많아졌으므로 가로 폭을 넓게 조정
    margin=dict(l=100, r=50, t=140, b=100),
    xaxis=dict(
        title=None, 
        showgrid=False,
        tickangle=45, # 라벨 겹침 방지를 위해 각도 조절
        dtick="M3"    # 3개월 단위로 눈금 표시
    ),
    yaxis=dict(
        title="평균 실거래가 (Average Transaction Price)",
        tickmode='array',
        tickvals=tick_vals,
        ticktext=tick_texts,
        showgrid=True,
        gridcolor='rgba(200, 200, 200, 0.3)',
        gridwidth=1,
        griddash='dot',
        range=[min_val * 0.8, max_val * 1.2]
    ),
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5)
)

fig.show()

In [17]:
import pandas as pd
import plotly.graph_objects as go

# 1. 데이터 준비 및 상승/하락 분류 (기존 로직 유지)
df_ana = df[(df['계약일'].dt.year >= 2023) & (df['계약일'].dt.year <= 2025)].copy()
df_ana = df_ana.sort_values(['건물명', '건물면적', '계약일'])

df_ana['직전거래가'] = df_ana.groupby(['건물명', '건물면적'])['물건금액'].shift(1)
df_ana['가격차이'] = df_ana['물건금액'] - df_ana['직전거래가']

def classify_trade(diff):
    if pd.isna(diff): return None
    if diff > 0: return '상승'
    elif diff < 0: return '하락'
    else: return '보합'

df_ana['거래유형'] = df_ana['가격차이'].apply(classify_trade)
df_ana['연월'] = df_ana['계약일'].dt.strftime('%Y-%m')

# 2. 월별 비중 계산
type_counts = df_ana.groupby(['연월', '거래유형']).size().unstack(fill_value=0)
type_ratio = type_counts.div(type_counts.sum(axis=1), axis=0) * 100

# 3. 시각화 생성
fig = go.Figure()

# 상승 거래 비중 (hovertemplate 추가)
fig.add_trace(go.Scatter(
    x=type_ratio.index, y=type_ratio['상승'],
    name='상승거래 비중',
    mode='lines+markers',
    line=dict(color='red', width=3),
    fill='tozeroy',
    fillcolor='rgba(255, 0, 0, 0.05)',
    # %{y:.2f}를 통해 소수점 2자리 제한
    hovertemplate='<b>상승거래 비중</b><br>연월: %{x}<br>비중: %{y:.2f}%<extra></extra>'
))

# 하락 거래 비중 (hovertemplate 추가)
fig.add_trace(go.Scatter(
    x=type_ratio.index, y=type_ratio['하락'],
    name='하락거래 비중',
    mode='lines+markers',
    line=dict(color='blue', width=2, dash='dot'),
    hovertemplate='<b>하락거래 비중</b><br>연월: %{x}<br>비중: %{y:.2f}%<extra></extra>'
))

# 4. 연도 시작점 실선(0~100) 및 연도 텍스트 (기존 유지)
years_to_mark = ['2024-01', '2025-01']
for year_mark in years_to_mark:
    if year_mark in type_ratio.index:
        fig.add_shape(
            type="line", x0=year_mark, x1=year_mark, y0=0, y1=100,
            line=dict(color="rgba(128, 128, 128, 0.6)", width=2, dash="solid")
        )
        fig.add_annotation(
            x=year_mark, y=100, text=f"<b>{year_mark.split('-')[0]}년</b>",
            showarrow=False, yshift=15, font=dict(size=14, color="#555555"),
            xanchor="center"
        )

# 5. 주요 정책 시점 주석 (기존 유지)
policy_events = [
    {'date': '2023-01', 'text': '1.3 대책'},
    {'date': '2024-09', 'text': '스트레스 DSR 2단계'},
    {'date': '2025-07', 'text': '스트레스 DSR 3단계'},
    {'date': '2025-10', 'text': '규제지역 확대'}
]

for event in policy_events:
    if event['date'] in type_ratio.index:
        fig.add_annotation(
            x=event['date'], y=type_ratio.loc[event['date'], '상승'],
            text=event['text'], showarrow=True, arrowhead=2,
            ax=0, ay=-45, font=dict(size=10, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="lightgrey"
        )

# 6. 레이아웃 설정
tick_vals = type_ratio.index
tick_texts = [f"{int(m.split('-')[1])}월" for m in tick_vals]

fig.update_layout(
    title=dict(
        text='<b>2023-2025 강남 3구 상승/하락 거래 비중 분석</b>',
        x=0.5, y=0.92, font=dict(size=22)
    ),
    xaxis=dict(title=None, tickangle=0, tickmode='array', tickvals=tick_vals, ticktext=tick_texts, showgrid=False),
    yaxis=dict(title="거래 비중 (%)", range=[0, 115], gridcolor='rgba(200, 200, 200, 0.2)', zeroline=False),
    template='plotly_white', height=650, width=1200,
    margin=dict(t=120, b=80),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()

상승거래 비중 (Up-Trade Proportion)

- "사람들이 이전 가격보다 더 비싸게 살 의지가 있는가?"를 뜻합니다.
- 동일 단지, 동일 면적의 직전 거래가보다 높은 가격에 체결된 거래의 비율입니다.
- 해석 방법:
- 50% 이상: 시장이 매우 뜨겁습니다. 매수자가 매도자의 호가를 적극적으로 수용하는 **'매도자 우위 시장(Seller's Market)'**입니다.

하락거래 비중 (Down-Trade Proportion)

-" 급하게 팔아야 하는 매도자가 얼마나 많은가?"를 뜻합니다.
- 직전 거래가보다 낮은 가격에 체결된 거래의 비율입니다.
- 해석 방법:
- 비중 증가: 금리 부담이나 세금 문제 등으로 인해 가격을 낮춰서라도 팔려는 **'급매물'**이 소화되고 있다는 신호입니다.
-상승 비중 추월 (Dead Cross): 하락 비중이 상승 비중보다 높아지면 본격적인 **'하락 전환'**의 신호탄으로 해석합니다.

In [18]:
import pandas as pd
import plotly.graph_objects as go

# 1. 데이터 준비 및 상승/하락 분류
df_ana = df[(df['계약일'].dt.year >= 2023) & (df['계약일'].dt.year <= 2025)].copy()
df_ana = df_ana.sort_values(['자치구명', '건물명', '건물면적', '계약일'])

# 가격 차이 계산 (자치구별 그룹화)
df_ana['직전거래가'] = df_ana.groupby(['자치구명', '건물명', '건물면적'])['물건금액'].shift(1)
df_ana['가격차이'] = df_ana['물건금액'] - df_ana['직전거래가']

def classify_trade(diff):
    if pd.isna(diff): return None
    if diff > 0: return '상승'
    elif diff < 0: return '하락'
    else: return '보합'

df_ana['거래유형'] = df_ana['가격차이'].apply(classify_trade)
df_ana['연월'] = df_ana['계약일'].dt.strftime('%Y-%m')

# 2. 분석 대상 자치구 및 설정
gu_list = ['강남구', '서초구', '송파구']
colors = {'강남구': '#E31A1C', '서초구': '#FF7F00', '송파구': '#33A02C'}

# 주요 정책 시점 정의
policy_events = [
    {'date': '2023-01', 'text': '1.3 대책'},
    {'date': '2024-09', 'text': '스트레스 DSR 2'},
    {'date': '2025-07', 'text': '스트레스 DSR 3'},
    {'date': '2025-10', 'text': '규제지역 확대'}
]

for gu in gu_list:
    gu_df = df_ana[df_ana['자치구명'] == gu]
    
    # 월별 비중 계산
    type_counts = gu_df.groupby(['연월', '거래유형']).size().unstack(fill_value=0)
    for col in ['상승', '하락']:
        if col not in type_counts.columns: type_counts[col] = 0
    type_ratio = type_counts.div(type_counts.sum(axis=1), axis=0) * 100
    
    # 3. 시각화 생성
    fig = go.Figure()

    # 상승 거래 비중 (호버 소수점 2자리 적용)
    fig.add_trace(go.Scatter(
        x=type_ratio.index, y=type_ratio['상승'],
        name=f'{gu} 상승비중',
        mode='lines+markers',
        line=dict(color=colors[gu], width=3),
        fill='tozeroy',
        fillcolor='rgba(200, 200, 200, 0.05)',
        hovertemplate='<b>%{x} 상승비중</b><br>비중: %{y:.2f}%<extra></extra>'
    ))

    # 하락 거래 비중
    fig.add_trace(go.Scatter(
        x=type_ratio.index, y=type_ratio['하락'],
        name=f'{gu} 하락비중',
        mode='lines+markers',
        line=dict(color='grey', width=2, dash='dot'),
        hovertemplate='<b>%{x} 하락비중</b><br>비중: %{y:.2f}%<extra></extra>'
    ))

    # 4. 연도 시작점 실선 및 연도 텍스트 (0~100 높이 제한)
    years_to_mark = ['2024-01', '2025-01']
    for year_mark in years_to_mark:
        if year_mark in type_ratio.index:
            fig.add_shape(
                type="line", x0=year_mark, x1=year_mark, y0=0, y1=100,
                line=dict(color="rgba(128, 128, 128, 0.5)", width=2)
            )
            fig.add_annotation(
                x=year_mark, y=100, text=f"<b>{year_mark.split('-')[0]}년</b>",
                showarrow=False, yshift=15, font=dict(size=14, color="#555555")
            )

    # 5. 정책 시점 주석 추가 (구별 상승 비중 데이터 포인트에 배치)
    for event in policy_events:
        if event['date'] in type_ratio.index:
            fig.add_annotation(
                x=event['date'], y=type_ratio.loc[event['date'], '상승'],
                text=event['text'], showarrow=True, arrowhead=2,
                ax=0, ay=-45, font=dict(size=10, color="black"),
                bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="lightgrey"
            )

    # 6. 레이아웃 및 X축 설정 ('1월', '2월'...)
    tick_vals = type_ratio.index
    tick_texts = [f"{int(m.split('-')[1])}월" for m in tick_vals]

    fig.update_layout(
        title=dict(
            text=f'<b>2023-2025 {gu} 상승거래 비중 및 정책 효과 분석</b>',
            x=0.5, y=0.92, font=dict(size=22)
        ),
        xaxis=dict(title=None, tickmode='array', tickvals=tick_vals, ticktext=tick_texts, showgrid=False),
        yaxis=dict(title="거래 비중 (%)", range=[0, 115], gridcolor='rgba(200, 200, 200, 0.2)', zeroline=False),
        template='plotly_white', height=600, width=1150,
        margin=dict(t=120, b=80),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()